In [ ]:
import os
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from interfaces import Environment, Agent, Simulator
from simulator import NuPlan
from environment import OmniRe
from setup import OmniReSetup
from agent import RandomAgent, SSRLAgent
from planners.oscillating_planner import OscillatingPlanner
from evaluators.evaluator import Evaluator
from evaluators.simple_evaluator import SimpleEvaluator
from path_utils import use_path
from image_utils import save_rgb_images_to_video, print_rgb_image
from state_types import Action, EnvState
import numpy as np
import copy
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner, PlannerInput
from nuplan.planning.simulation.planner.simple_planner import SimplePlanner
from nuplan.common.actor_state.waypoint import Waypoint

In [ ]:
env_setup = None

# Now use the context manager for clean path handling
with use_path("drivestudio", True):
    # Define paths relative to the drivestudio directory
    relative_config_path = "configs/datasets/nuplan/8cams_undistorted.yaml"
    relative_checkpoint_path = (
        "output/master-project/run_omnire_undistorted_8cams_0"
    )

    print(f"Working directory: {os.getcwd()}")
    print(f"Config path (relative to drivestudio): {relative_config_path}")
    print(f"Absolute config path: {os.path.abspath(relative_config_path)}")

    # Check if these files exist in this context
    if not os.path.exists(relative_config_path):
        print(
            f"ERROR: Config file not found at {os.path.abspath(relative_config_path)}"
        )
    if not os.path.exists(relative_checkpoint_path):
        print(
            f"ERROR: Checkpoint directory not found at {os.path.abspath(relative_checkpoint_path)}"
        )

    # Only initialize if files exist
    if os.path.exists(relative_config_path) and os.path.exists(
        relative_checkpoint_path
    ):
        env_setup = OmniReSetup(relative_config_path, relative_checkpoint_path)
        print("Successfully initialized OmniRe environment model")
    else:
        print("Failed to initialize environment model due to missing files")



In [ ]:
def simulate(
    simulator: Simulator, 
    env: Environment, 
    planner: AbstractPlanner, 
    n_steps: int, 
    evaluator: Evaluator,
) -> list[EnvState]:
    sensor_outputs: list[Env_state] = []
    evaluations = []

    # ADD EVALUATION FUNCTION IN THIS LOOP
    for i in tqdm(range(n_steps), desc="Running simulate:"):
        # Exit loop if simulator is not running
        if not simulator.simulation.is_simulation_running():
            break

        # Setup
        state = simulator.get_state()

        # Environment
        sensor_output: Env_state = env.get_sensor_output(state)
        sensor_outputs.append(sensor_output)

        # Planner
        current_input = simulator.get_planner_input()
        trajectory: InterpolatedTrajectory = planner.compute_planner_trajectory(current_input)

        # Evaluator
        history = current_input.history
        scenario = simulator.scenario
        evaluation = evaluator.compute_cumulative_score(history, scenario)

        current_ego_state = trajectory.get_sampled_trajectory()[0]

        # Simulator
        simulator.do_action(trajectory)

    return sensor_outputs, tire_steering_angles, car_headings, velocities, accelerations

In [ ]:
simulator = NuPlan()
env = OmniRe(env_setup)
# planner = SimplePlanner(10.0, 1.0, np.array([0.0, 0.0]), 5.0, 0.0)
planner = OscillatingPlanner(horizon_seconds=1.0, sampling_time=1.0, max_steering_angle=math.pi/8, steering_angle_increment=0.2)
evaluator = SimpleEvaluator()
n_steps = 300

with open("output.txt", "w") as f:
    f.write("")

def main():
    """Main function to run the simulation.""" 
    sensor_outputs, tire_steering_angles, car_headings, velocities, accelerations = simulate(simulator, env, planner, n_steps, evaluator)
    x = np.arange(0, 0.05 * n_steps, 0.05)
    plt.plot(x, tire_steering_angles, label="Tire Steering Angles")
    plt.plot(x, car_headings, label="Headings")
    plt.plot(x, velocities, label="Velocities")
    plt.plot(x, accelerations, label="Accelerations")

    plt.title("Vehicle Dynamics Over Time")
    plt.xlabel("Time [s]")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()
    save_rgb_images_to_video(sensor_outputs, "output_video.mp4")

main()